In [1]:
library("forecast")
library("stringr")
library("ggplot2")
library("MLmetrics")

Warning message:
"package 'MLmetrics' was built under R version 3.5.2"
Attaching package: 'MLmetrics'

The following object is masked from 'package:base':

    Recall



In [2]:
setwd("C:\\Users\\kevin\\ziknet-trends-rolling\\data\\Colombia\\processed_data")

In [3]:
files = list.files(".", pattern="*.csv")
files

[1] "huila_2016-2017.csv"           "santander_2016-2017.csv"      
[3] "santander_norte_2016-2017.csv" "tolima_2016-2017.csv"         
[5] "valle_cauca_2016-2017.csv"

In [4]:
myfiles = lapply(files, function(i){
    read.csv(i, header=TRUE)
})
print(myfiles)

[[1]]
          Date Searches Cases
1   03/01/2016       82   334
2   10/01/2016       68   473
3   17/01/2016       19     0
4   24/01/2016       53  1295
5   31/01/2016       45   834
6   07/02/2016      100   572
7   14/02/2016       61   453
8   21/02/2016       41   416
9   28/02/2016       44   290
10  06/03/2016       12   264
11  13/03/2016       58   197
12  20/03/2016       26   118
13  27/03/2016       20   117
14  03/04/2016       20   112
15  10/04/2016       53    85
16  17/04/2016       20    73
17  24/04/2016       42    55
18  01/05/2016       31    62
19  08/05/2016        0    73
20  15/05/2016        0   111
21  22/05/2016        0   -26
22  29/05/2016        0    36
23  05/06/2016       13    28
24  12/06/2016       29    25
25  19/06/2016        0    18
26  26/06/2016        0    16
27  03/07/2016        2    17
28  10/07/2016        0    21
29  17/07/2016      100    20
30  24/07/2016       42     7
31  31/07/2016       14    10
32  07/08/2016        0     7
33  

In [5]:
setwd("C:\\Users\\kevin\\ziknet-trends-rolling\\Colombia\\SARIMA\\1Week")

In [6]:
myplot <- function(dataset, filename) {
    print("-----")
    print(filename)
    split_index = nrow(dataset)/2
    
    window_size = 4
    lag_weeks = 1

    train = dataset[1:(split_index - lag_weeks + 1),2:3]
    test = dataset[(split_index+1):(nrow(dataset)), 2:3]

    endIndex = nrow(test)
    trainIndex = nrow(train)
    
    predictions = numeric(0)

    for (index in 1:endIndex){
        fit <- auto.arima(dataset$Cases[1:trainIndex+index-1], seasonal=TRUE, xreg=dataset$Searches[1:trainIndex+index-1])
        
        #Forecast regressors
        fit_reg = auto.arima(dataset$Searches[1:trainIndex+index-1])
        reg_forecast <- forecast(fit_reg, h=lag_weeks)
        reg_forecast_frame <- data.frame(reg_forecast)
        reg_forecast_point_forecast <- reg_forecast_frame$Point.Forecast
        reg_df = dataset$Searches[1:trainIndex+index-1]
        l = 1
        reg_length = length(reg_df)
        
        while(l <= lag_weeks) {
            reg_df[reg_length + l] = reg_df[l]
            l = l + 1
        }
        
        #Forecast
        
#         print(arimaorder(fit))
        fcast <- forecast(fit, h=lag_weeks, xreg = reg_df)
        forecasted_data <- data.frame(fcast)
        predictions[index] <- forecasted_data$Point.Forecast[lag_weeks]
    }
    
    length(predictions)
    print(filename)
#     output = cat("1Week", filename)
    write.csv(predictions, file=filename)
    mse = MSE(predictions, test$Cases)
    print(mse)
    
}

In [7]:
fileIndex = 1
for(file in myfiles) {
    myplot(file, files[fileIndex])
    fileIndex = fileIndex + 1
}

[1] "-----"
[1] "huila_2016-2017.csv"
[1] "huila_2016-2017.csv"
[1] 67.78095
[1] "-----"
[1] "santander_2016-2017.csv"
[1] "santander_2016-2017.csv"
[1] 70.66905
[1] "-----"
[1] "santander_norte_2016-2017.csv"
[1] "santander_norte_2016-2017.csv"
[1] 427.6772
[1] "-----"
[1] "tolima_2016-2017.csv"
[1] "tolima_2016-2017.csv"
[1] 164.5859
[1] "-----"
[1] "valle_cauca_2016-2017.csv"
[1] "valle_cauca_2016-2017.csv"
[1] 858.0404
